<br>

This is how to do it with fasterai !

In [ ]:
#| include: false
from fastai.vision.all import *
from fasterai.misc.all import *
from nbdev.showdoc import *

import torch
import torch.nn as nn
import torch.nn.functional as F

1. Get the data

In [ ]:
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")

def label_func(f): return f[0].isupper()

dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(64))

2. Train the model

In [ ]:
learn = Learner(dls, resnet18(num_classes=2), metrics=accuracy)
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,time
0,0.615641,0.596630,0.688092,00:10
1,0.582679,0.558671,0.689445,00:10
2,0.529308,0.517995,0.744926,00:10
3,0.481804,0.449941,0.784168,00:10
4,0.400030,0.414093,0.800406,00:10


3. Fold !

In [ ]:
#| include: false
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

In [ ]:
bn = BN_Folder()
new_model = bn.fold(learn.model)

The batch norm layers have been replaced by an Identity layer, and the weights of the convolutions have been modified accordingly.

In [ ]:
new_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): Identity()
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Identity()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Identity()
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(128, 

We can see that the new model possess fewer parameters

In [ ]:
count_parameters(learn.model)

11177538

In [ ]:
count_parameters(new_model)

11172738

But is also faster to run ! 

In [ ]:
x,y = dls.one_batch()

In [ ]:
%%timeit
learn.model(x[0][None].cuda())

5.59 ms ± 547 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
new_model(x[0][None].cuda())

4.14 ms ± 446 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


But most importantly, has the exact same perfomance as before:

In [ ]:
new_learn = Learner(dls, new_model, metrics=accuracy)

In [ ]:
new_learn.validate()

(#2) [0.4140927791595459,0.8004059791564941]